In [23]:
import math

In [24]:
def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

In [25]:
def find_Hs(W, U, alpha0, A0, alphaZ, Az, d_h, d_r, d_z): 
    """
    Calculates the primary beam scattered from the primary barrier.

    Parameters:
    W (float): Workload (Gy/wk) for the worst-case Wall G.
    U (float): Usage factor for the worst-case Wall G.
    alpha0 (float): Albedo of the primary barrier, dependent on material, energy, 
                    angle of incidence, and reflection.
    A0 (float): Dependent on field size and d_h (m^2).
    alphaZ (float): Albedo of Wall Z, dependent on material, energy, 
                    angle of incidence, and reflection.
    Az (float): Area of the inner maze entrance as viewed from the center of the 
                primary barrier projected onto Wall Z (m^2).
    d_h (float): Distance from the source to the proximal surface of the primary barrier (m).
    d_r (float): Distance from the center of the primary barrier, past the edge of IMW, 
                 to point B (m).
    d_z (float): Distance from the point B to the plane of the door (m).
    """

    H_s = (W * U * alpha0 * A0 * alphaZ * Az) / (d_h * d_r * d_z)**2
    return H_s


def find_Hps(W, U, a, F, alpha1, A1, d_sca, d_sec, d_zz): 
    """

    Parameters:
    
    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    a : scatter fraction dependent on scatter angle and beam energy 
    F : field size (cm^2)
    alpha1 : albedo of A1. Dependent on material, 0.5 MeV, angle of incidence, angle of reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sca : distance from source to iso (m)
    d_sec : distance from iso to center of A1 (m)
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ps = (W * U * a * (F/400)* alpha1 * A1) / (d_sca * d_sec * d_zz)**2
    return H_ps


def find_Hls(W, U, C, alpha_ls, A1, d_sec, d_zz):
    """
    
    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    U : usage factor for worst case Wall G
    C : IMRT factor (unitless)
    alpha_ls : albedo of A1, dependent on material, Co-60
               angle of incidence, and reflection is 0 deg
    A1 : area of the wall at the end of the maze visible from the door (m^2). Width of A1 x maze height
    d_sec : distance from iso to center of A1 (m) if gantry rotates uniformly
    d_zz : distance from center of A1 to plane of door (m)
    """

    H_ls = (10**-3 *W * U * C * alpha_ls * A1) / (d_sec * d_zz)**2
    return H_ls


def find_Hlt(W, C, U, t_slant, TVL1, TVLe, d_L):
    """

    Parameters:

    W : workload (Gy/wk) for worst case Wall G
    C : IMRT factor (unitless)
    U : usage factor for worst case Wall G
    t_slant : slant thickness of IMW (cm)
    TVL1 : TVL of IMW material (cm). Dependent on incident energy
    TVLe : equilibrium TVL of IMW material (cm). Dependent on incident energy
    d_L : distance from source to center of plane of door (m)
    """

    H_lt = (10**-3 * W * C * U * 10**-(1 + (t_slant - TVL1) / TVLe)) / d_L**2
    return H_lt


In [26]:
P = 1
U = 0.25 # use factor 
T = 1

# maze width
maze_width = convert_to_meters(8,5.00242)
# maze height
maze_height = convert_to_meters(10,0)
# inner maze height
inner_maze_height = convert_to_meters(9,0)
# dz
dz = convert_to_meters(24,11.10850)
# dzz
dzz = convert_to_meters(34,0.99985)

test1 = dz / (maze_width * maze_height)**0.5

test2 = dzz / (maze_width * maze_height)**0.5

if test1 < 6 and test1 > 2:
    print('PASS')

if test2 < 6 and test2 > 2:
    print('PASS')

if 1 < maze_height / maze_width and maze_height / maze_width < 2:
    print('PASS')
    print(maze_height/maze_width)

print(f"maze height = {maze_height} m")
print(f"maze width = {maze_width} m")
print(f"inner maze height = {inner_maze_height} m")


PASS
PASS
PASS
1.188090344765997
maze height = 3.048 m
maze width = 2.5654614679999996 m
inner maze height = 2.7432 m


In [ ]:
W_6MV_noQA = 500 # Gy/wk
W_18MV = 250 # Gy/wk

# H_s
# Variables for Vault #1
d_r = convert_to_meters(25,1.28538)
d_h = convert_to_meters(17,9.04728) + 1
area_0 = (0.40 * d_h)**2 # m
alhpa_z = 13e-3 # Albedo of Wall Z, concrete, 0.5 MeV, 0 deg incidence, 60 deg reflection
# Find A_z
width_Az = convert_to_meters(7,9.78277)
area_z = width_Az * inner_maze_height

print("H_s")
print("--------------------------")
print(f"dz = {dz} m")
print(f"dzz = {dzz} m")
print(f"d_r = {d_r} m")
print(f"d_h = {d_h} m")
print(f"width_Az = {width_Az} m")
print("--------------------------")
print(f"area_0 = {area_0} m^2")
print(f"area_z = {area_z} m^2")

H_s_6MV_noQA = find_Hs(W_6MV_noQA, 0.25, 0.5, area_0, 0.5, area_z, d_h, d_r, dz)
H_s_18MV = find_Hs(W_18MV, 0.25, 0.25, area_0, 0.5, area_z, d_h, d_r, dz)

print("---------------------------")
print(f"H_s_6MV_noQA = {H_s_6MV_noQA} Gy/wk")
print(f"H_s_18MV = {H_s_18MV} Gy/wk")
print("--------------------------")

H_s
--------------------------
dz = 7.5973559 m
dzz = 10.38859619 m
d_r = 7.652648652000001 m
d_h = 6.4114009119999995 m
width_Az = 2.3820823580000003 m
--------------------------
area_0 = 6.57696986470311 m^2
area_z = 6.534528324465601 m^2
---------------------------
H_s_6MV_noQA = 0.009665754429167263 Gy/wk
H_s_18MV = 0.004832877214583631 Gy/wk
--------------------------


In [44]:
W_6MV_wQA = 520 # Gy/wk
# Workload for H_ps is the same as H_s

# H_ps
d_sec = convert_to_meters(31,6.75193)
d_sca = 1 # m
a_6MV = 1.39e-3
a_18MV = 8.64e-4
alpha_1_ps = 22e-3 # albedo of A1, concrete, 0.5 MeV, 45 deg incidence, 0 deg reflection
width_A1 = convert_to_meters(10,11.97257)
area_1 = maze_height * width_A1 # m^2

print("H_ps")
print("--------------------------")
print(f"d_zz = {dzz} m")
print(f"d_sec = {d_sec} m")
print(f"d_sca = {d_sca} m")
print("--------------------------")
print(f"a_6MV = {a_6MV}")
print(f"a_18MV = {a_18MV}")
print("---------------------------")
print(f"width_A1 = {width_A1} m")
print(f"area_1 = {area_1} m^2")

H_ps_6MV = find_Hps(W_6MV_wQA, 0.25, 1600, a_6MV, alpha_1_ps, area_1, d_sca, d_sec, dzz)
H_ps_18MV = find_Hps(W_18MV, 0.25, 1600, a_18MV, alpha_1_ps, area_1, d_sca, d_sec, dzz)

print("---------------------------")
print(f"H_ps_6MV = {H_ps_6MV} Gy/wk")
print(f"H_ps_18MV = {H_ps_18MV} Gy/wk")
print("---------------------------")

H_ps_tot = H_ps_6MV + H_ps_18MV
print(f"H_ps_tot = {H_ps_tot} Gy/wk")


H_ps
--------------------------
d_zz = 10.38859619 m
d_sec = 9.620299022000001 m
d_sca = 1 m
--------------------------
a_6MV = 0.00139
a_18MV = 0.000864
---------------------------
width_A1 = 3.3521032780000004 m
area_1 = 10.217210791344002 m^2
---------------------------
H_ps_6MV = 1.6266057571359304e-05 Gy/wk
H_ps_18MV = 4.8609137180597805e-06 Gy/wk
---------------------------
H_ps_tot = 2.1126971289419086e-05 Gy/wk


In [ ]:
W_L_6MV = 1990 # Gy/wk
W_L_18MV = 250 # Gy/wk

# H_lt
d_leak = convert_to_meters(26,4.88130) # m
thickness_IMW = 27.94 # cm
t_slant = thickness_IMW / math.cos(math.radians(29.74)) # cm

# TVL for 6 MV and 18 MV
tvl1_6MV = 34 # cm
tvle_6MV = 29 # cm
tvl1_18MV = 36 # cm
tvle_18MV = 34 # cm

# B factors for 6 MV and 18 MV
B_IMW_6MV = B2(t_slant, tvl1_6MV, tvle_6MV)
B_IMW_18MV = B2(t_slant, tvl1_18MV, tvle_18MV)
print("H_lt")
print("--------------------------")
print(f"d_leak = {d_leak} m")
print(f"t_slant = {t_slant} cm")
print(f"B_IMW_6MW = {B_IMW_6MV}")
print(f"B_IMW_18MV = {B_IMW_18MV}")

H_Lt_6MV = find_Hlt(1990, 1, 0.25, t_slant, tvl1_6MV, tvle_6MV, d_leak)
H_Lt_18MV = find_Hlt(250, 1, 0.25, t_slant, tvl1_18MV, tvle_18MV, d_leak)
H_lt_tot = H_Lt_6MV + H_Lt_18MV
print
print(f"H_Lt_6MV = {H_Lt_6MV} Gy/wk")
print(f"H_Lt_18MV = {H_Lt_18MV} Gy/wk")
print("--------------------------")
print(f"H_lt_tot = {H_lt_tot} Gy/wk")


H_lt
--------------------------
d_leak = 8.04878502 m
t_slant = 32.17835951453385 cm
B_IMW_6MW = 0.11556203717315604
B_IMW_18MV = 0.1295391940676817
H_Lt_6MV = 0.0008874576119735566 Gy/wk
H_Lt_18MV = 0.0001249742539002665 Gy/wk
--------------------------
H_lt_tot = 0.001012431865873823 Gy/wk
